Locate a Satellite

In [ ]:
import requests # send POST data with HTTP request
import ephem # friendly orbital calculations
from pandas import read_csv # parse csv from URL response into searchable object
from datetime import datetime, timedelta
!python --version

Python 3.7.11


In [ ]:
# For this task we need to hit the SpaceTrack API so you'll need to go to
# https://www.space-track.org/auth/createAccount and create a login
USERNAME = "<email>"
PASSWORD = "<password>"

## Utility Functions

In [ ]:
# don't know the code for an object you want to look at?
# no worries! let's look at what we have available.

# you can have a poke around yourself
def fetch_satcat():
    df = read_csv("https://celestrak.com/pub/satcat.csv")
    # remove objects no longer in orbit
    df = df[df.DECAY_DATE.isnull()]
    return df

# or search for an object by name/details you know
def search_satcat(value, attribute="OBJECT_NAME", cat=None):
    df = cat if cat is not None else fetch_satcat()
    if attribute in df.columns:
        return df[df[attribute] == value].iloc[0] # first match
    return None

## Step-by-Step

In [ ]:
# PICK AN OBJECT YOU WANT TO TRACK

# I've picked Vanguard 1 (the oldest know object still in orbit)
OBJECT_NAME = "ISS (ZARYA)"

# Let's find its attributes, particularly its NORAD ID
attributes = search_satcat(OBJECT_NAME)
norad_id = attributes.NORAD_CAT_ID

# You can use this to have a look at the information available for your object
attributes

OBJECT_NAME         ISS (ZARYA)
OBJECT_ID             1998-067A
NORAD_CAT_ID              25544
OBJECT_TYPE                 PAY
OPS_STATUS_CODE               +
OWNER                       ISS
LAUNCH_DATE          1998-11-20
LAUNCH_SITE               TYMSC
DECAY_DATE                  NaN
PERIOD                    92.97
INCLINATION               51.64
APOGEE                      421
PERIGEE                     419
RCS                     399.052
DATA_STATUS_CODE            NaN
ORBIT_CENTER                 EA
ORBIT_TYPE                  ORB
Name: 25543, dtype: object

In [ ]:
# GET MOST RECENT DETECTION OF THAT OBJECT

def fetch_most_recent_detection(norad_id):
    query = ("https://www.space-track.org/basicspacedata/query/class/"
                f"gp_history/NORAD_CAT_ID/{norad_id}/orderby/EPOCH%20desc/"
                f"limit/1/format/3le")
    request = "https://www.space-track.org/ajaxauth/login"

    # we have to include some variables into the POST request header
    print(f"Fetching most recent detection of object {norad_id}...")
    creds = {"identity": USERNAME, "password": PASSWORD, "query": query}
    response = requests.post(request, data=creds)

    # check for failure response
    print(f"RESPONSE: {response.reason} ({response.status_code})")
    # (HTTP success codes are 200-299)
    if response.status_code // 100 != 2: return None

    return response.text

# Now actually do the thing
detection = fetch_most_recent_detection(norad_id)
print(f"Most recent detection:\n\n{detection}")

Fetching most recent detection of object 25544...
RESPONSE: OK (200)
Most recent detection:

0 ISS (ZARYA)
1 25544U 98067A   21231.85950031  .00001072  00000-0  27662-4 0  9997
2 25544  51.6440  27.8463 0001517 333.1303 175.6838 15.48918110298467



In [ ]:
# CALCULATE OBJECT POSITION AT EPOCH IN EARTH-CENTRIC CO-ORDINATES

name, line1, line2 = detection.strip().split("\n")
sat = ephem.readtle(name, line1, line2)
sat.compute() # computes propagation for current moment in time

def dms_to_decimal_degrees(dms):
    string_val = f"{dms}"
    d, m, s = tuple(string_val.split(":"))
    num_d = float(d)
    negative = (num_d < 0)
    dd = abs(num_d) + (float(m) / 60) + (float(s) / 360)
    return dd if not negative else -dd

lat = dms_to_decimal_degrees(sat.sublat)
lon = dms_to_decimal_degrees(sat.sublong)
print(f"Object should currently be at {lat}°N, {lon}°E")

Object should currently be at -4.609999999999999°N, -163.34027777777777°E


In [ ]:
# CALCULATE FUTURE TRAJECTORY GIVEN VARIOUS FORCES

minutes_in_future = 60
EPOCH_FORMAT = '%Y-%m-%d %H:%M:%S.%f'
new_time = datetime.now() + timedelta(minutes=minutes_in_future)
new_time_string = new_time.strftime(EPOCH_FORMAT)
new_epoch = ephem.date(new_time_string)
sat.compute(new_epoch)

lat = dms_to_decimal_degrees(sat.sublat)
lon = dms_to_decimal_degrees(sat.sublong)
print(f"Object should be at {lat}°N, {lon}°E at {new_time_string}")

Object should be at 41.93694444444444°N, 43.062222222222225°E at 2021-08-20 03:58:50.735546
